<a href="https://colab.research.google.com/github/Charank18/Charank18/blob/main/Copy_of_Welcome_To_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import pandas as pd


In [5]:
df=pd.read_csv('final_labels.csv')

In [6]:
df.head()

,entry_id,link_id,parent_id,entry_utc,subreddit,author,body,image,label_date,week,group,sheet_order,level_1,level_2,level_3,strength,highlight,split
0,exoxn7,t3_exoxn7,NaN,1580652620,badwomensanatomy,doggodone,Do you have the skin of a 80 year old grandma?...,Yes,17-02-2020,1,1,"(1,)",Nonmisogynistic,None_of_the_categories,NaN,NaN,NaN,train
1,fgb3bdv,t3_exoxn7,t3_exoxn7,1580658139,badwomensanatomy,Machaeon,This is taking a grain of truth and extrapolat...,NaN,17-02-2020,1,1,"(1, 1)",Nonmisogynistic,None_of_the_categories,NaN,NaN,NaN,train
2,fgc6tlu,t3_exoxn7,t3_exoxn7,1580669695,badwomensanatomy,CuniculusVincitOmnia,Honestly my favorite thing about this is that ...,NaN,17-02-2020,1,1,"(1, 2)",Nonmisogynistic,None_of_the_categories,NaN,NaN,NaN,test
3,fge6msg,t3_exoxn7,t1_fgc6tlu,1580692566,badwomensanatomy,its331am,Source? Doesnt sound right to me idk,NaN,17-02-2020,1,1,"(1, 2, 1)",Nonmisogynistic,None_of_the_categories,NaN,NaN,NaN,test
4,fgawus5,t3_exoxn7,t3_exoxn7,1580656280,badwomensanatomy,JaxDefore,"Damn, I saw a movie in which the old woman bat...",NaN,17-02-2020,1,1,"(1, 3)",Misogynistic,Derogation,Moral_inferiority,Nature of the abuse is Implicit,old woman bathed in the blood if virgins,train


In [ ]:
df.columns

Index(['entry_id', 'link_id', 'parent_id', 'entry_utc', 'subreddit', 'author',
       'body', 'image', 'label_date', 'week', 'group', 'sheet_order',
       'level_1', 'level_2', 'level_3', 'strength', 'highlight', 'split'],
      dtype='object')

In [7]:
import re
import nltk
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split

# Ensure stopwords are downloaded
nltk.download('stopwords')
nltk.download('punkt_tab')


stop_words = set(stopwords.words('english'))

def clean_text(text):
    if pd.isna(text):
        return ""  # Handle missing values in text
    text = text.lower()  # Convert to lowercase
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)  # Remove special characters
    words = nltk.word_tokenize(text)  # Tokenize words
    words = [word for word in words if word not in stop_words]  # Remove stopwords
    return ' '.join(words)

# Load dataset (assuming df is already loaded)
df['body'] = df['body'].apply(clean_text)

# Drop unnecessary columns
drop_cols = ['entry_id', 'link_id', 'parent_id', 'entry_utc', 'label_date', 'week', 'sheet_order', 'image']
df = df.drop(columns=drop_cols, errors='ignore')

# Handling categorical values if necessary
df['subreddit'] = df['subreddit'].astype('category').cat.codes
df['author'] = df['author'].astype('category').cat.codes
df['split'] = df['split'].astype('category')

# Convert labels to numerical values
label_mapping = {label: idx for idx, label in enumerate(df['level_1'].unique())}
df['labels'] = df['level_1'].map(label_mapping)

# Splitting dataset for training
df_train = df[df['split'] == 'train']
df_test = df[df['split'] == 'test']


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [8]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 20.0 MB/s eta 0:00:00


In [9]:
from sklearn.model_selection import train_test_split
from transformers import AutoModelForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments
from datasets import Dataset

In [11]:
import torch
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
def tokenize_function(examples):
    return tokenizer(examples["body"], padding=True, truncation=True, max_length=512)

train_dataset = Dataset.from_pandas(df_train[['body', 'labels']])
test_dataset = Dataset.from_pandas(df_test[['body', 'labels']])

train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

# Load transformer model
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=len(label_mapping))

# Enable GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    fp16=torch.cuda.is_available(),  # Enable mixed precision if GPU is available
)

# Trainer setup
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
)

# Train the model
trainer.train()

# Save model and tokenizer
model.save_pretrained("./misogyny_model")
tokenizer.save_pretrained("./misogyny_model")

print("Model training complete. Saved trained model.")


Map:   0%|          | 0/5264 [00:00<?, ? examples/s]

Map:   0%|          | 0/1303 [00:00<?, ? examples/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-11-385c2c9616b1>:34: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss
1,No log,0.214257
2,0.231900,0.192965
3,0.231900,0.212298


Model training complete. Saved trained model.


In [12]:
import torch
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from textblob import TextBlob
import gensim.downloader as api
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset

# Download NLTK resources
nltk.download('stopwords')
nltk.download('punkt')

# Load Word2Vec for joke detection
word2vec_model = api.load("word2vec-google-news-300")
stop_words = set(stopwords.words('english'))

# Load pre-trained sarcasm and toxicity models
sarcasm_tokenizer = AutoTokenizer.from_pretrained("cardiffnlp/twitter-roberta-base-sarcasm")
sarcasm_model = AutoModelForSequenceClassification.from_pretrained("cardiffnlp/twitter-roberta-base-sarcasm")

toxicity_tokenizer = AutoTokenizer.from_pretrained("unitary/toxic-bert")
toxicity_model = AutoModelForSequenceClassification.from_pretrained("unitary/toxic-bert")

def clean_text(text):
    """Clean and preprocess text"""
    if pd.isna(text):
        return ""
    text = text.lower()
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    words = nltk.word_tokenize(text)
    words = [word for word in words if word not in stop_words]
    return ' '.join(words)

def detect_sarcasm(text):
    """Detect sarcasm using a pre-trained model"""
    inputs = sarcasm_tokenizer(text, return_tensors="pt", truncation=True, max_length=512)
    outputs = sarcasm_model(**inputs)
    prediction = torch.argmax(outputs.logits, dim=1).item()
    return "sarcasm" if prediction == 1 else "not_sarcasm"

def detect_toxicity(text):
    """Detect toxicity/harmful intent using a BERT-based model"""
    inputs = toxicity_tokenizer(text, return_tensors="pt", truncation=True, max_length=512)
    outputs = toxicity_model(**inputs)
    prediction = torch.sigmoid(outputs.logits).tolist()[0]
    return "harmful_intent" if any(p > 0.5 for p in prediction) else "not_harmful"

def detect_joke(text):
    """Detect jokes using sentiment analysis and word embeddings"""
    sentiment = TextBlob(text).sentiment.polarity
    joke_words = ["funny", "hilarious", "joke", "lol", "lmao", "rofl"]
    words = text.lower().split()
    humor_score = sum(1 for w in words if any(word2vec_model.similarity(w, jw) > 0.5 for jw in joke_words if w in word2vec_model))
    return "joke" if sentiment > 0.5 or humor_score > 1 else "not_joke"

def detect_context(text):
    """Classify text into sarcasm, joke, harmful intent, misogyny, or non-misogyny"""
    if detect_sarcasm(text) == "sarcasm":
        return "sarcasm"
    elif detect_joke(text) == "joke":
        return "joke"
    elif detect_toxicity(text) == "harmful_intent":
        return "harmful_intent"
    return "misogyny" if "misogyny" in text.lower() else "non-misogyny"

# Load dataset (assuming df is loaded)
df["body"] = df["body"].apply(clean_text)
df["context_label"] = df["body"].apply(detect_context)

# Map context labels to numbers
label_mapping = {"misogyny": 0, "non-misogyny": 1, "sarcasm": 2, "joke": 3, "harmful_intent": 4}
df["labels"] = df["context_label"].map(label_mapping)

# Split dataset
df_train = df[df["split"] == "train"]
df_test = df[df["split"] == "test"]

# Tokenization for BERT training
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

def tokenize_function(examples):
    return tokenizer(examples["body"], padding="max_length", truncation=True, max_length=512)

# Convert to Hugging Face dataset format
train_dataset = Dataset.from_pandas(df_train[["body", "labels"]])
test_dataset = Dataset.from_pandas(df_test[["body", "labels"]])

train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

# Load Transformer model
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=len(label_mapping))

# Use GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,  # Increased epochs for better generalization
    weight_decay=0.01,
    logging_dir="./logs",
    fp16=torch.cuda.is_available(),
    load_best_model_at_end=True
)

# Trainer setup
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
)

# Train model
trainer.train()

# Save trained model
model.save_pretrained("./misogyny_model")
tokenizer.save_pretrained("./misogyny_model")

print("Training complete. Model saved.")


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


[==================================================] 100.0% 1662.8/1662.8MB downloaded


OSError: cardiffnlp/twitter-roberta-base-sarcasm is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `huggingface-cli login` or by passing `token=<your_token>`

In [ ]:
#TESTING

In [13]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import numpy as np

def load_model(model_path="./misogyny_model"):
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    model = AutoModelForSequenceClassification.from_pretrained(model_path)
    model.eval()  # Set to evaluation mode
    return tokenizer, model

def predict(text, tokenizer, model, label_mapping):
    inputs = tokenizer(text, padding=True, truncation=True, max_length=512, return_tensors="pt")
    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits
    probabilities = torch.nn.functional.softmax(logits, dim=-1).cpu().numpy()
    predicted_label_idx = np.argmax(probabilities)
    predicted_label = [label for label, idx in label_mapping.items() if idx == predicted_label_idx][0]
    return predicted_label, probabilities

# Load model and tokenizer
tokenizer, model = load_model()

# Define label mapping (ensure this matches the training phase)
label_mapping = {"non-misogynistic": 0, "misogynistic": 1}  # Update with actual mappings

# Example test input
text = "Women should stay in the kitchen."

# Predict
predicted_label, probabilities = predict(text, tokenizer, model, label_mapping)
print(f"Predicted Label: {predicted_label}")
print(f"Probabilities: {probabilities}")

Predicted Label: non-misogynistic
Probabilities: [[0.510041   0.48995894]]


In [31]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import numpy as np

def load_model(model_path="./misogyny_model"):
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    model = AutoModelForSequenceClassification.from_pretrained(model_path)
    model.eval()  # Set to evaluation mode
    return tokenizer, model

def predict(text, tokenizer, model, label_mapping):
    inputs = tokenizer(text, padding=True, truncation=True, max_length=512, return_tensors="pt")
    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits
    probabilities = torch.nn.functional.softmax(logits, dim=-1).cpu().numpy()
    predicted_label_idx = np.argmax(probabilities)
    predicted_label = [label for label, idx in label_mapping.items() if idx == predicted_label_idx][0]
    return predicted_label, probabilities

# Load model and tokenizer
tokenizer, model = load_model()

# Define label mapping (ensure this matches the training phase)
label_mapping = {"non-misogynistic": 0, "misogynistic": 1}  # Update with actual mappings

# Example test input
text = "that women is good"

# Predict
predicted_label, probabilities = predict(text, tokenizer, model, label_mapping)
print(f"Predicted Label: {predicted_label}")
print(f"Probabilities: {probabilities}")

Predicted Label: non-misogynistic
Probabilities: [[0.9721577  0.02784228]]


In [32]:
!pip install huggingface_hub


In [33]:
from huggingface_hub import notebook_login

notebook_login()


In [34]:
from huggingface_hub import HfApi

api = HfApi()
api.create_repo(repo_id="misogyny_model", private=False)


RepoUrl('https://huggingface.co/Charankarnati18/misogyny_model', endpoint='https://huggingface.co', repo_type='model', repo_id='Charankarnati18/misogyny_model')

In [35]:
# ✅ Step 1: Upload ZIP file from Desktop
from google.colab import files

# This will open a file picker to select your ZIP file
uploaded = files.upload()

# ✅ Step 2: Get the file name (it will automatically detect)
import os

# Extract the uploaded file name
file_name = list(uploaded.keys())[0]
file_path = f"/content/{file_name}"

print(f"File '{file_name}' uploaded successfully!")

# ✅ Step 3: Install HuggingFace Hub library
!pip install huggingface_hub

# ✅ Step 4: Upload the ZIP file to HuggingFace
from huggingface_hub import HfApi

# Initialize HuggingFace API
api = HfApi()

# ✅ Step 5: Provide HuggingFace Repo Details
your_username = "Charankarnati18"  # 🔥 Replace this with your HuggingFace username
repo_name = "misogyny_model"     # 🔥 Replace this with your HuggingFace repo name

# ✅ Step 6: Upload ZIP to HuggingFace
api.upload_file(
    path_or_fileobj=file_path,
    path_in_repo=file_name,
    repo_id=f"{your_username}/{repo_name}"
)

print(f"✅ File '{file_name}' uploaded successfully to HuggingFace!")
print(f"🚀 Access it here: https://huggingface.co/{your_username}/{repo_name}")


Saving misogyny_model.zip to misogyny_model.zip
File 'misogyny_model.zip' uploaded successfully!


misogyny_model.zip:   0%|          | 0.00/247M [00:00<?, ?B/s]

✅ File 'misogyny_model.zip' uploaded successfully to HuggingFace!
🚀 Access it here: https://huggingface.co/Charankarnati18/misogyny_model


In [36]:
from transformers import pipeline

# Load the model from Hugging Face
classifier = pipeline('text-classification', model='Charankarnati18/misogyny_model')

# Provide input
input_text = "Your input text here"

# Get the model's prediction
result = classifier(input_text)
print(result)

ValueError: Could not load model Charankarnati18/misogyny_model with any of the following classes: (<class 'transformers.models.auto.modeling_auto.AutoModelForSequenceClassification'>, <class 'transformers.models.auto.modeling_tf_auto.TFAutoModelForSequenceClassification'>). See the original errors:

while loading with AutoModelForSequenceClassification, an error is thrown:
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/transformers/pipelines/base.py", line 289, in infer_framework_load_model
    model = model_class.from_pretrained(model, **kwargs)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/transformers/models/auto/auto_factory.py", line 567, in from_pretrained
    raise ValueError(
ValueError: Unrecognized configuration class <class 'transformers.models.deprecated.nat.configuration_nat.NatConfig'> for this kind of AutoModel: AutoModelForSequenceClassification.
Model type should be one of A

In [38]:
import zipfile
import os

# Path to the .zip file
zip_path = "misogyny_model.zip"

# Directory to extract the files to
extract_dir = "misogyny_model_extracted"

# Create the directory if it doesn't exist
os.makedirs(extract_dir, exist_ok=True)

# Extract the .zip file
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

print(f"Model files extracted to: {extract_dir}")

Model files extracted to: misogyny_model_extracted


In [39]:
from transformers import AutoModel, AutoTokenizer

# Path to the extracted model directory
model_dir = "misogyny_model_extracted"

# Load the model and tokenizer
model = AutoModel.from_pretrained(model_dir)
tokenizer = AutoTokenizer.from_pretrained(model_dir)

# Preprocess the input
input_text = "Your input text here"
inputs = tokenizer(input_text, return_tensors="pt")

# Run inference
outputs = model(**inputs)
print(outputs)

BaseModelOutput(last_hidden_state=tensor([[[-0.2701, -1.0294,  0.5148,  ..., -1.0807, -0.7573,  1.1519],
         [-0.2727, -0.7683,  1.0194,  ..., -0.8608, -0.7429,  0.7694],
         [-0.2341, -0.0739,  0.9831,  ..., -1.3671, -0.6685,  0.8196],
         [ 0.1333, -0.2977,  0.7732,  ..., -1.2376, -0.7088,  0.9446],
         [-0.1694, -0.7359,  0.6512,  ..., -0.8669, -0.7997,  0.6832],
         [ 0.9221,  0.0381, -0.3116,  ..., -0.0273, -0.6416, -0.1977]]],
       grad_fn=<NativeLayerNormBackward0>), hidden_states=None, attentions=None)


In [40]:
from transformers import AutoModel, AutoTokenizer

# Path to the extracted model directory
model_dir = "misogyny_model_extracted"

# Load the model and tokenizer
model = AutoModel.from_pretrained(model_dir)
tokenizer = AutoTokenizer.from_pretrained(model_dir)

# Preprocess the input
input_text = "Your input text here"
inputs = tokenizer(input_text, return_tensors="pt")

# Run inference
outputs = model(**inputs)
print(outputs)

BaseModelOutput(last_hidden_state=tensor([[[-0.2701, -1.0294,  0.5148,  ..., -1.0807, -0.7573,  1.1519],
         [-0.2727, -0.7683,  1.0194,  ..., -0.8608, -0.7429,  0.7694],
         [-0.2341, -0.0739,  0.9831,  ..., -1.3671, -0.6685,  0.8196],
         [ 0.1333, -0.2977,  0.7732,  ..., -1.2376, -0.7088,  0.9446],
         [-0.1694, -0.7359,  0.6512,  ..., -0.8669, -0.7997,  0.6832],
         [ 0.9221,  0.0381, -0.3116,  ..., -0.0273, -0.6416, -0.1977]]],
       grad_fn=<NativeLayerNormBackward0>), hidden_states=None, attentions=None)


In [41]:
from huggingface_hub import upload_folder

# Path to the folder you want to upload
folder_path = "misogyny_model_extracted"

# Upload the folder to the repository
upload_folder(
    folder_path=folder_path,
    repo_id="Charankarnati18/misogyny_model",  # Repository name
    repo_type="model",  # Type of repository (model, dataset, etc.)
)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Charankarnati18/misogyny_model/commit/ad2faba4295ea49bdc8c1ac86286afe3b02bd82c', commit_message='Upload folder using huggingface_hub', commit_description='', oid='ad2faba4295ea49bdc8c1ac86286afe3b02bd82c', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Charankarnati18/misogyny_model', endpoint='https://huggingface.co', repo_type='model', repo_id='Charankarnati18/misogyny_model'), pr_revision=None, pr_num=None)

In [45]:
from transformers import AutoModel, AutoTokenizer
import torch
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained("Charankarnati18/misogyny_model")
# Replace with your model's repository name
model_name = "Charankarnati18/misogyny_model"

# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Example input text
input_text = "women is good"

# Tokenize the input
inputs = tokenizer(input_text, return_tensors="pt")

# Run inference
with torch.no_grad():
    outputs = model(**inputs)

# Example: For a classification model, get the predicted class
predicted_class = torch.argmax(outputs.logits, dim=-1)
print(f"Predicted class: {predicted_class}")

Predicted class: tensor([0])
